# Mise à disposition du jeu de données stroke_dataset via une API REST

## Description du dataset

Le jeu de données utilisé provient de Kaggle : [Stroke Prediction Dataset](https://www.kaggle.com/datasets/fedesoriano/stroke-prediction-dataset).  
Il contient des données de patients avec différentes caractéristiques médicales et sociales, ainsi que l'information si le patient a subi un accident vasculaire cérébral (AVC) ou non.

Télécharger les données et ajouter les dans un dossier data/.

Les colonnes des données sont :  
- `id` : Identifiant unique du patient  
- `gender` : Sexe  
- `age` : Âge  
- `hypertension` : Présence d'hypertension (0 ou 1)  
- `heart_disease` : Présence de maladie cardiaque (0 ou 1)  
- `ever_married` : Statut marital  
- `work_type` : Type d'emploi  
- `Residence_type` : Urbaine ou rurale  
- `avg_glucose_level` : Moyenne du taux de glucose  
- `bmi` : Indice de masse corporelle  
- `smoking_status` : Statut tabagique  
- `stroke` : Présence d'AVC (0 ou 1)

## Projet

Vous devez exposer les données patients du jeu de données via une API REST afin que les données soit utilisables par d'autres équipes (médecins, data science, étude, etc.).

Cette API REST sera développée avec FastAPI et les spécifications sont les suivantes :
| Méthode | Endpoint                                      | Fonctionnalité                                                                                                    |
| ------- | --------------------------------------------- | ----------------------------------------------------------------------------------------------------------------- |
| `GET`   | `/patients/{id}`                              | Récupère les détails d’un patient donné (via son identifiant unique)                                              |
| `GET`   | `/patients?stroke=1&gender=Female&max_age=60` | Renvoie les patients filtrés selon plusieurs critères : AVC (oui/non), genre, âge maximal                         |
| `GET`   | `/stats/`                                     | Fournit des statistiques agrégées sur les patients (ex. : nb total de patients, âge moyen, taux d’AVC, répartition hommes/femmes, etc.) |



---

## Quelques définitions


1. Qu’est-ce qu’une API REST ?

- API signifie Application Programming Interface (Interface de Programmation d’Application). C’est un ensemble de règles et de protocoles qui permettent à des logiciels de communiquer entre eux.
- REST signifie Representational State Transfer. C’est un style architectural pour concevoir des services web.
Il en existe d'autres mais REST est celui que vous rencontrerez le plus souvent.
- Vous avez utilisé une API REST via l'API Google Books.

- A quoi sert une API REST ?

    - Permet à différentes applications de communiquer facilement, même si elles sont écrites dans des langages différents.
    - Permet d’accéder à des services distants (ex : bases de données, services web) de manière standardisée.
    - Facilite la création d’applications modulaires et évolutives (front-end, back-end, mobile, etc.)

2. Principes clés d’une API REST

- a. Utilisation du protocole HTTP
Les échanges entre client et serveur utilisent des méthodes HTTP standard comme :

    - GET : pour récupérer des données
    - POST : pour envoyer ou créer des données
    - PUT : pour mettre à jour des données
    - DELETE : pour supprimer des données

- b. Accès aux ressources via des URLs

Chaque ressource (par exemple un livre, un utilisateur) est accessible via une URL unique.

Exemple fictif:
    https://api.example.com/books/123 pour accéder au livre d’identifiant 123.

- c. Stateless (sans état)

Le serveur ne conserve aucune information sur le client entre deux requêtes. Chaque requête doit contenir toutes les informations nécessaires.

- d. Représentations des données

Les données sont envoyées et reçues généralement en format JSON ou XML, qui sont faciles à lire et à manipuler.

- e. Utilisation de codes status HTTP

Chaque réponse du serveur est accompagnée d’un code HTTP indiquant le résultat de la requête. (cf [liste des codes](https://developer.mozilla.org/en-US/docs/Web/HTTP/Reference/Status))

## Outils utilisés
1. FastAPI

FastAPI est un framework Python moderne, rapide et très utilisé dans le milieu professionnel pour construire des API REST. 

Il permet :
- une définition simple des routes et des paramètres  
- La génération automatique de documentation interactive (Swagger UI)  
- FastAPI lit les requêtes entrantes, les traite avec ton code Python, et retourne une réponse HTTP (en JSON).

2. Uvicorn : exécute l'application FastAPI

- Uvicorn est un serveur ASGI (Asynchronous Server Gateway Interface) : c'est une interface standard pour gérer les requêtes de manière asynchrone et performante, notamment utile pour les applications modernes.
- Il attend les requêtes HTTP (par exemple depuis un navigateur), les transmet à FastAPI, et renvoie la réponse.
- Uvicorn permet à l'API de fonctionner : sans Uvicorn ou un autre serveur, FastAPI ne peut pas fonctionner.


3. Swagger UI : l’interface de doc et test interactive

- Swagger UI est généré automatiquement par FastAPI.
- C’est une interface web qui permet de :
    - Voir toutes les routes disponibles dans l'API
    - Tester les routes en envoyant des requêtes sans écrire de code (bouton try it out)
    - Voir les paramètres attendus et les formats de réponse
    
4. Résumé des interactions

- Tester la route de base de l'API grâce à la commande :
```bash
    poetry run fastapi dev stroke_api/main.py
```

--> Qu'est-ce qu'il se passe derrière cette commande ?

- Uvicorn démarre un serveur local
- FastAPI génère automatiquement une interface : Swagger UI, accessible sur http://127.0.0.1:8000/docs qui affiche toutes les routes définies dans le code python FastAPI
- Quand on clique sur "Try it out" dans Swagger UI, Swagger envoie une requête HTTP au serveur (ici Uvicorn)
- Le serveur (Uvicorn) la reçoit, l’envoie à FastAPI, qui traite et renvoie une réponse
- Swagger UI affiche la réponse de l’API (par ex : liste de patients)

---

Import des bibliothèques utiles au projet

In [ ]:
import pandas as pd
from fastapi import FastAPI
import numpy as np


## 1. Prétraitement des données / Data preprocessing

Les données réelles sont rarement prêtes à être utilisées directement. Elles peuvent contenir des erreurs, des valeurs manquantes, des doublons, des formats incohérents, ou ne pas être adaptées au modèle ou au système cible.

Le prétraitement consiste à nettoyer, structurer et transformer les données brutes avant de les exploiter dans un projet (modèle IA, API, visualisation, etc.).

Vous avez déjà prétraité des données, petit rappel des éléments sur lesquels travailler dans un prétraitment classique et les méthodes pandas qu'il est possible d'utiliser pour les différentes étapes (des exeples d'utilisation des méthodes pandas sont disponibles dans la doc) : 
- explorer les données pour identifier les types de données, valeurs manquantes, incohérence ([info](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.info.html), [dtypes](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dtypes.html))
- adapter les types si nécessaire ([astypes](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.astype.html))
- identifier les doublons et les supprimer s'il y en a ([duplicated](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.duplicated.html), [drop_duplicates](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.drop_duplicates.html))
- traiter les valeurs manquantes s'il y en a ([fillna](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.fillna.html), [dropna](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.dropna.html), [replace](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.replace.html))
- identifier les incohérences éventuelles (valeurs aberrantes/outliers) en vérifiant si les valeurs min, max, moyennes sont raisonnables (recherche internet si nécessaire) ([describe](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.describe.html)), et les traiter.
- Traiter les valeurs aberrantes si vous en détectez ([loc](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.loc.html) pour récupérer les lignes qui répondent à une certaine condition, cf exemple ci-dessous)


**Exemple df.loc :**

Récupérer toutes les lignes de df telles que la valeur de "nom de colonne" >= 0

```df_subset = df.loc[stroke_data_df['nom de colonne'] >= 0]```



---
### **TODO**
1.a. Prétraiter les données du dataset.


1.b. Documenter dans le README.md :
- Les étapes de prétraitement,
- Justification des choix concernant le traitement des valeurs manquantes (si besoin),
- Liste des valeurs raisonnables utilisées pour détecter les valeurs aberrantes, 
- Justification des choix pour traiter les valeurs aberrantes (si besoin).

2.a. Chercher des infos sur le format de fichier parquet et indiquer les sources consultées : 
- Différence principale avec le format csv ?
- Dans quels cas l'utiliser ? 
- Pourquoi c'est un format adapté aux gros volumes de données ?

Différence principale avec le format csv ? </br>,
Le format parquet transforme les données en binaire, compresse les données (donc moins volumineux), optimal pour traiter un grand nombre de données.
Dans quels cas l'utiliser ? </br>,
Principalement pour le stockage de données tabulaires.
Pourquoi c'est un format adapté aux gros volumes de données ? </br>,
Parce qu'il compresse les données.

2.b. Sauvegarder les données prétraiteées dans un fichier parquet ([to_parquet](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.to_parquet.html)).


In [10]:
# Prétraitement de données
import pandas as pd
df = pd.read_csv("data/healthcare-dataset-stroke-data-cleaned.csv")
df.head()


,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [15]:
df.info()
df.describe()



<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5110 entries, 0 to 5109
Data columns (total 12 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   id                 5110 non-null   int64  
 1   gender             5110 non-null   object 
 2   age                5110 non-null   float64
 3   hypertension       5110 non-null   int64  
 4   heart_disease      5110 non-null   int64  
 5   ever_married       5110 non-null   object 
 6   work_type          5110 non-null   object 
 7   Residence_type     5110 non-null   object 
 8   avg_glucose_level  5110 non-null   float64
 9   bmi                4909 non-null   float64
 10  smoking_status     5110 non-null   object 
 11  stroke             5110 non-null   int64  
dtypes: float64(3), int64(4), object(5)
memory usage: 479.2+ KB


,id,age,hypertension,heart_disease,avg_glucose_level,bmi,stroke
count,5110.000000,5110.000000,5110.000000,5110.000000,5110.000000,4909.000000,5110.000000
mean,36517.829354,43.226614,0.097456,0.054012,106.147677,28.893237,0.048728
std,21161.721625,22.612647,0.296607,0.226063,45.283560,7.854067,0.215320
min,67.000000,0.080000,0.000000,0.000000,55.120000,10.300000,0.000000
25%,17741.250000,25.000000,0.000000,0.000000,77.245000,23.500000,0.000000
50%,36932.000000,45.000000,0.000000,0.000000,91.885000,28.100000,0.000000
75%,54682.000000,61.000000,0.000000,0.000000,114.090000,33.100000,0.000000
max,72940.000000,82.000000,1.000000,1.000000,271.740000,97.600000,1.000000


In [18]:
import pandas as pd

# Chargement du fichier
df = pd.read_csv("data/healthcare-dataset-stroke-data-cleaned.csv")

#  Afficher le nombre de doublons
print("Nombre de lignes dupliquées :", df.duplicated().sum())

#  Supprimer les doublons
df = df.drop_duplicates()

#  Afficher les premières lignes après nettoyage
print(df.head())



Nombre de lignes dupliquées : 0
      id  gender   age  hypertension  heart_disease ever_married  \
0   9046    Male  67.0             0              1          Yes   
1  51676  Female  61.0             0              0          Yes   
2  31112    Male  80.0             0              1          Yes   
3  60182  Female  49.0             0              0          Yes   
4   1665  Female  79.0             1              0          Yes   

       work_type Residence_type  avg_glucose_level   bmi   smoking_status  \
0        Private          Urban             228.69  36.6  formerly smoked   
1  Self-employed          Rural             202.21   NaN     never smoked   
2        Private          Rural             105.92  32.5     never smoked   
3        Private          Urban             171.23  34.4           smokes   
4  Self-employed          Rural             174.12  24.0     never smoked   

   stroke  
0       1  
1       1  
2       1  
3       1  
4       1  


In [19]:
print(f"Nombre de lignes avant suppression : {len(df)}")
df = df.drop_duplicates()
print(f"Nombre de lignes après suppression : {len(df)}")




Nombre de lignes avant suppression : 5110
Nombre de lignes après suppression : 5110


In [4]:
  # Export de la copie csv en dataframe
df = pd.read_csv('data/healthcare-dataset-stroke-data-cleaned.csv')


# Fonction pour l'imputation de la catégorie d'âge
def impute_age_category(age):
    if age < 13:
        return 'enfant'
    elif age < 18:
        return 'ado'
    elif age <= 45:
        return 'adulte_21_45'
    elif age <= 65:
        return 'adulte_46_65'
    else:
        return 'senior'

# Création colonne & imputation des patients selon leur âge dans une catégorie d'âge
df['age_category'] = df['age'].apply(impute_age_category)

# AJOUT : imputation du statut fumeur pour les enfants
df.loc[df['age_category'] == 'enfant', 'smoking_status'] = 'never smoked'

# Calcul de la médiane de l'IMC selon la catégorie d'âge
bmi_medians = df.groupby('age_category')['bmi'].median()

# Fonction pour catégoriser le taux de glycémie
def impute_glucose_category(glucose):
    if glucose < 90:
        return 'faible'
    elif glucose <= 125:
        return 'normal'
    else:
        return 'élevé'

# Création colonne & imputation dans les catégories (glycémie)
df['glucose_category'] = df['avg_glucose_level'].apply(impute_glucose_category)

#selon le genre, statut SP, catégorie d'âge et taux de glycémie
bmi_medians_multi = (
    df.groupby(['gender', 'work_type', 'age_category', 'glucose_category'])['bmi']
    .median()
    .round(1)
    .reset_index()
)


# Fusion avec le dataset nettoyé
df = df.merge(
    bmi_medians_multi,
    on=['gender', 'work_type', 'age_category', 'glucose_category'],
    how='left',
    suffixes=('', '_impute')
)

# Remplacement des valeurs manquantes
df['bmi'] = df['bmi'].fillna(df['bmi_impute'])
df.drop(columns=['bmi_impute'], inplace=True)

# Export et vérification
print('Valeur manquantes restantes: ', df['bmi'].isna().sum())
df = df.to_csv('df_final_test.csv')
df = pd.read_csv('df_final_test.csv')

Valeur manquantes restantes:  0


In [7]:
import pandas as pd

df = pd.read_csv("df_final_test.csv")

#  Nombre de valeurs manquantes par colonne
print(df.isnull().sum())

#  En pourcentage 
print((df.isnull().sum() / len(df)) * 100)


Unnamed: 0           0
id                   0
gender               0
age                  0
hypertension         0
heart_disease        0
ever_married         0
work_type            0
Residence_type       0
avg_glucose_level    0
bmi                  0
smoking_status       0
stroke               0
age_category         0
glucose_category     0
dtype: int64
Unnamed: 0           0.0
id                   0.0
gender               0.0
age                  0.0
hypertension         0.0
heart_disease        0.0
ever_married         0.0
work_type            0.0
Residence_type       0.0
avg_glucose_level    0.0
bmi                  0.0
smoking_status       0.0
stroke               0.0
age_category         0.0
glucose_category     0.0
dtype: float64


In [41]:
# BMI extrêmes
bmi_out = df[(df["bmi"] > 60) | (df["bmi"] < 12)]
print(bmi_out)

# Age <1 an
age_out = df[df["age"] < 1]
print(age_out)


      Unnamed: 0     id  gender    age  hypertension  heart_disease  \
270          270  72911  Female  57.00             1              0   
358          358  66333    Male  52.00             0              0   
466          466   1307  Female  61.00             1              0   
544          544    545    Male  42.00             0              0   
928          928  41097  Female  23.00             1              0   
1559        1559  37759  Female  53.00             0              0   
1609        1609  38043  Female   1.24             0              0   
2128        2128  56420    Male  17.00             1              0   
2187        2187  59993    Male  40.00             0              0   
2764        2764  20292  Female  24.00             0              0   
2840        2840  65895  Female  52.00             0              0   
3307        3307   3205  Female  79.00             0              0   
3825        3825  72784  Female  52.00             0              0   
4188  

In [42]:
# Filtrer les enfants (moins de 18 ans)
enfants = df[df["age"] < 1]

# Trier par BMI décroissant
enfants_bmi_haut = enfants.sort_values(by="bmi", ascending=False)

# Afficher les 5 enfants avec le BMI le plus élevé
print(enfants_bmi_haut.head())


      Unnamed: 0     id  gender   age  hypertension  heart_disease  \
2801        2801    760    Male  0.80             0              0   
363          363   7559  Female  0.64             0              0   
2490        2490  48406    Male  0.88             0              0   
1206        1206  68908  Female  0.72             0              0   
2481        2481  20257    Male  0.88             0              0   

     ever_married work_type Residence_type  avg_glucose_level   bmi  \
2801           No  children          Urban              75.22  33.1   
363            No  children          Urban              83.82  24.9   
2490           No  children          Urban              85.38  23.4   
1206           No  children          Urban              66.36  23.0   
2481           No  children          Urban              90.62  22.4   

     smoking_status  stroke age_category glucose_category  
2801   never smoked       0       enfant           faible  
363    never smoked       0     

In [43]:
# Supprimer les lignes où "gender" est "Other"
df = df[df["gender"] != "Other"]

# Vérifier les valeurs restantes
print(df["gender"].unique())

['Male' 'Female']


-----
## Développement de l'API

A présent que les données sont propres, on peut débuter la création de l'API.

Pour cela, vous allez avoir besoin de quelques fonctions permettant de filtrer les données.

Vous allez les définir ci-dessous, ce qui vous permettra de les tester puis les fonctions seront reportées dans le fichier filters.py.

## Route `/patients/`
- Cette route retourne une liste filtrée de patients
- On souhaite pouvoir filtrer par `gender`, `stroke` ou `max_age`

L'objectif est ici de définir une fonction python qui prend en entrée les paramètres optionnels : _gender_, *stroke*, *max_age* et qui renvoie un dictionnaire filtré des données.

On décompose la rédaction de cette fonction en plusieurs étapes. 

Dans un premier temps, écrire et tester les filtres que l'on souhaite appliquer sur les données (utiliser [loc](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.loc.html))

In [44]:
# Filtrer le dataframe pour ne garder que les patients pour lesquels "stroke=1"
df= pd.read_csv('df_final_test.csv')



In [45]:

stroke = df.loc[df['stroke'] == 1]
print(stroke)

     Unnamed: 0     id  gender   age  hypertension  heart_disease  \
0             0   9046    Male  67.0             0              1   
1             1  51676  Female  61.0             0              0   
2             2  31112    Male  80.0             0              1   
3             3  60182  Female  49.0             0              0   
4             4   1665  Female  79.0             1              0   
..          ...    ...     ...   ...           ...            ...   
244         244  17739    Male  57.0             0              0   
245         245  49669  Female  14.0             0              0   
246         246  27153  Female  75.0             0              0   
247         247  34060    Male  71.0             1              0   
248         248  43424  Female  78.0             0              0   

    ever_married      work_type Residence_type  avg_glucose_level   bmi  \
0            Yes        Private          Urban             228.69  36.6   
1            Yes  Sel

In [46]:
# Filtrer les données pour ne garder que les patients pour lesquels "gender="male"
gender = df.loc[df['gender'] == 'Male']
print(gender)

      Unnamed: 0     id gender   age  hypertension  heart_disease  \
0              0   9046   Male  67.0             0              1   
2              2  31112   Male  80.0             0              1   
5              5  56669   Male  81.0             0              0   
6              6  53882   Male  74.0             1              1   
13            13   8213   Male  78.0             0              1   
...          ...    ...    ...   ...           ...            ...   
5097        5097  64520   Male  68.0             0              0   
5098        5098    579   Male   9.0             0              0   
5099        5099   7293   Male  40.0             0              0   
5100        5100  68398   Male  82.0             1              0   
5108        5108  37544   Male  51.0             0              0   

     ever_married      work_type Residence_type  avg_glucose_level   bmi  \
0             Yes        Private          Urban             228.69  36.6   
2             Yes  

In [10]:
# Filtrer les données pour ne garder que les patients tels que "age <= max_age"

max_age = df ['age'].max()
print(max_age)
df_max_age = df.loc[df['age'] <= max_age]
print(df_max_age)


82.0
      Unnamed: 0     id  gender   age  hypertension  heart_disease  \
0              0   9046    Male  67.0             0              1   
1              1  51676  Female  61.0             0              0   
2              2  31112    Male  80.0             0              1   
3              3  60182  Female  49.0             0              0   
4              4   1665  Female  79.0             1              0   
...          ...    ...     ...   ...           ...            ...   
5105        5105  18234  Female  80.0             1              0   
5106        5106  44873  Female  81.0             0              0   
5107        5107  19723  Female  35.0             0              0   
5108        5108  37544    Male  51.0             0              0   
5109        5109  44679  Female  44.0             0              0   

     ever_married      work_type Residence_type  avg_glucose_level   bmi  \
0             Yes        Private          Urban             228.69  36.6   
1 

Appliquer successivement les 3 filtres au sein d'une fonction qui prend en entrée le dataframe, _stroke_, _gender_, _max_age_ et qui renvoie une liste de dictionnaire de patients (utiliser la méthode pandas [to_dict](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_dict.html)).

Exemple
```
[{'id': 9046,
  'gender': 'Male',
  'age': 67.0,
  ...
  'smoking_status': 'formerly smoked',
  'stroke': 1},
 {'id': 31112,
  'gender': 'Male',
  'age': 80.0,
  ...
  'smoking_status': 'formerly smoked',
  'stroke': 1}]
  ```

In [28]:
#def filter_patient(df, max_age, gender, stroke):

def filter_patients(df: pd.DataFrame, gender: str, max_age: float, stroke: int) -> list[dict]:
    
    filtered_df = df[
        (df['stroke'] == stroke) &
        (df['gender'] == gender) &
        (df['age'] <= max_age)
    ]

    return filtered_df.to_dict(orient='records')

filtre1 = filter_patients(df, max_age= 80,gender= 'Male', stroke=1)
print(filtre1)


[{'Unnamed: 0': 0, 'id': 9046, 'gender': 'Male', 'age': 67.0, 'hypertension': 0, 'heart_disease': 1, 'ever_married': 'Yes', 'work_type': 'Private', 'Residence_type': 'Urban', 'avg_glucose_level': 228.69, 'bmi': 36.6, 'smoking_status': 'formerly smoked', 'stroke': 1, 'age_category': 'senior', 'glucose_category': 'élevé'}, {'Unnamed: 0': 2, 'id': 31112, 'gender': 'Male', 'age': 80.0, 'hypertension': 0, 'heart_disease': 1, 'ever_married': 'Yes', 'work_type': 'Private', 'Residence_type': 'Rural', 'avg_glucose_level': 105.92, 'bmi': 32.5, 'smoking_status': 'never smoked', 'stroke': 1, 'age_category': 'senior', 'glucose_category': 'normal'}, {'Unnamed: 0': 6, 'id': 53882, 'gender': 'Male', 'age': 74.0, 'hypertension': 1, 'heart_disease': 1, 'ever_married': 'Yes', 'work_type': 'Private', 'Residence_type': 'Rural', 'avg_glucose_level': 70.09, 'bmi': 27.4, 'smoking_status': 'never smoked', 'stroke': 1, 'age_category': 'senior', 'glucose_category': 'faible'}, {'Unnamed: 0': 13, 'id': 8213, 'gend

A présent on souhaite ajouter des informations sur les types des paramètres et valeurs de retour de la fonction pour faciliter sa compréhension et son utilisation, ce qu’on appelle l’annotation de type (type hinting).

Cette pratique facilite la lecture et la maintenance du code.

Quels changements pour la fonction ?

A la suite de chaque paramètre, on ajoute le type attendu pour le paramètre. À la suite des paramètres on ajoute le type de ce que qui est retourné par la fonction, dans l'exemple ici : 

```def filter_patient(stroke_data_df: pd.DataFrame, gender: str, etc) -> list[dict]```

Ajouter les types dans la définition de la fonction.




Tester la fonction en ne mettant pas de valeur pour *max_age*.

Que se passe-t-il ?

In [66]:
#def filter_patient(df, max_age, gender, stroke):

def filter_patients(df: pd.DataFrame, gender: str, max_age: float, stroke: int) -> list[dict]:
    
    filtered_df = df[
        (df['stroke'] == stroke) &
        (df['gender'] == gender) &
        (df['age'] <= max_age)
    ]

    return filtered_df.to_dict(orient='records')

filtre1 = filter_patients(df, max_age,gender= 'Male', stroke=1)
print(filtre1)


TypeError: filter_patients() got multiple values for argument 'gender'

Dans la fonction écrite ci-dessus, chaque paramètre est obligatoire. 

On souhaite pouvoir filtrer les patients sur 0, 1 ou 2 des paramètres de la fonction (filtrer seulement sur *max_age*  mais ne pas appliquer de filtres sur _gender_ et _stroke_ par exemple).

On peut rendre optionnel les paramètres d'un fonction en choisissant une valeur par défault. Si on utilise la fonction en n'utilisant pas ces paramètres alors la valeur par défault est utilisé.

Copier coller votre fonction ci-dessous et ajouter en paramètre : `max_age=None`

et ajouter la condition suivante **avant le filtre** sur `max_age` : 

```if max_age is not None : ``` 

Si la fonction _filter_patient_ est appelée sans argument *max_age*, alors le filtre sur *max_age* n'est pas appliqué. 

Il est tout à fait possible de définir une valeur par défault par exemple 30 ans : dans ce cas si la fonction est appelée sans argument *max_age*, alors par défault on filtre les patients ayant moins de 30 ans.

**ATTENTION :** Les paramètres optionnels doivent toujours être à la fin de la liste de paramètres.

In [69]:
def filter_patients(df: pd.DataFrame, gender: str, stroke: int, max_age: float = None) -> list[dict]:
    filtered_df = df[
        (df['stroke'] == stroke) &
        (df['gender'] == gender)
    ]

    if max_age is not None:
        filtered_df = filtered_df[filtered_df['age'] < max_age]

    return filtered_df.to_dict(orient='records')

# Appel de la fonction : patients de genre 'Female', stroke=0, moins de 30 ans
patients = filter_patients(df, gender='Female', stroke=0, max_age=30)

# Affichage
print("Patients filtrés :")
for patient in patients:
    print(patient)

Patients filtrés :
{'Unnamed: 0': 251, 'id': 16523, 'gender': 'Female', 'age': 8.0, 'hypertension': 0, 'heart_disease': 0, 'ever_married': 'No', 'work_type': 'Private', 'Residence_type': 'Urban', 'avg_glucose_level': 110.89, 'bmi': 17.6, 'smoking_status': 'never smoked', 'stroke': 0, 'age_category': 'enfant', 'glucose_category': 'normal'}
{'Unnamed: 0': 265, 'id': 19584, 'gender': 'Female', 'age': 20.0, 'hypertension': 0, 'heart_disease': 0, 'ever_married': 'No', 'work_type': 'Private', 'Residence_type': 'Urban', 'avg_glucose_level': 84.62, 'bmi': 19.7, 'smoking_status': 'smokes', 'stroke': 0, 'age_category': 'adulte_21_45', 'glucose_category': 'faible'}
{'Unnamed: 0': 274, 'id': 70336, 'gender': 'Female', 'age': 25.0, 'hypertension': 0, 'heart_disease': 0, 'ever_married': 'Yes', 'work_type': 'Private', 'Residence_type': 'Urban', 'avg_glucose_level': 60.84, 'bmi': 24.5, 'smoking_status': 'never smoked', 'stroke': 0, 'age_category': 'adulte_21_45', 'glucose_category': 'faible'}
{'Unname

In [70]:
# fonction filter_patient paramètre max_age optionnel
def filter_patients(df: pd.DataFrame, gender: str, stroke: int, max_age: float = None) -> list[dict]:
   
    filtered_df = df[
        (df['stroke'] == stroke) &
        (df['gender'] == gender) 
        
        
    ]
    
    if max_age is not None:
        filtered_df = filtered_df[filtered_df['age'] <= max_age]

    return filtered_df.to_dict(orient='records')
print("Patients filtrés :")
for patient in patients:
    print(patient)


Patients filtrés :
{'Unnamed: 0': 251, 'id': 16523, 'gender': 'Female', 'age': 8.0, 'hypertension': 0, 'heart_disease': 0, 'ever_married': 'No', 'work_type': 'Private', 'Residence_type': 'Urban', 'avg_glucose_level': 110.89, 'bmi': 17.6, 'smoking_status': 'never smoked', 'stroke': 0, 'age_category': 'enfant', 'glucose_category': 'normal'}
{'Unnamed: 0': 265, 'id': 19584, 'gender': 'Female', 'age': 20.0, 'hypertension': 0, 'heart_disease': 0, 'ever_married': 'No', 'work_type': 'Private', 'Residence_type': 'Urban', 'avg_glucose_level': 84.62, 'bmi': 19.7, 'smoking_status': 'smokes', 'stroke': 0, 'age_category': 'adulte_21_45', 'glucose_category': 'faible'}
{'Unnamed: 0': 274, 'id': 70336, 'gender': 'Female', 'age': 25.0, 'hypertension': 0, 'heart_disease': 0, 'ever_married': 'Yes', 'work_type': 'Private', 'Residence_type': 'Urban', 'avg_glucose_level': 60.84, 'bmi': 24.5, 'smoking_status': 'never smoked', 'stroke': 0, 'age_category': 'adulte_21_45', 'glucose_category': 'faible'}
{'Unname

In [ ]:
# test fonction sans argument max_age


Ajouter des valeurs par défault et les conditions pour chaque filtre.

Pour les types, on indique qu'il s'agit de paramètres optionels en utilisant le module python _typing_

```
from typing import Optional
def filter_patient(stroke_data_df: pd.DataFrame, gender: Optional[str] = None,etc)
```

Adapter les types en utilisant ce modèle.

In [ ]:
# fonction avec ajout de paramètres par défault et de type

Tester la fonction sans argument pour les filtres, elle doit donc renvoyer le dataframe non filtré.

In [ ]:
# test fonction sans argument pour les filtres

Cette fonction va être utilisée dans la définition de l'API pour créer une route qui permette d'accéder à des données filtrées sur les patients.

Dans le fichier de définition de l'API, toutes les fonctions vont travailler sur les données du fichier. 

Pour alléger les fonctions on va donc utiliser une **variable globale** pour les données et supprimer le paramètre `df` de la fonction.

On lit les données en début de fichier puis on travaille au sein des fonctions sur une copie du dataframe de données.


**En résumé les modifications à faire sont :**


- Supprimer le paramètre df de la fonction,
- Ajouter en début de fonction :  
```df = stroke_data_df.copy()```

1. Dans le fichier filters.py, il suffit d'ajouter : 
- lecture du fichier de données prétraitée dans la variable *df* en début de fichier (utiliser pandas),
- @app.get("/patients/") pour définir le route,
puis la fonction.

2. Dans le fichier api.py: appeler la fonction dans la route correspondante.

Tester la route avec 

```poetry run fastapi dev stroke_api/main.py```

http://127.0.0.1:8000/docs : utiliser la fonctionnalité Try it out pour tester la route.

---
## Autres routes

De la même manière, créer les fonctions appropriées pour la création de :
- la route `/patients/{id}` : Récupère les détails d’un patient donné (via son identifiant unique) 

- la route `/stats/` : Fournit des statistiques agrégées sur les patients (ex. : nb total de patients, âge moyen, taux d’AVC, répartition hommes/femmes).

- Lister les tâches à faire sous forme d'issue github : travailler sur une branche différentes pour l'ajout de chacune des routes.